# Demo

Reading JSON data of an existing coupled system

In [1]:
import os
import json


input_path = os.path.join(
    os.path.dirname(os.path.abspath('')),
    'mixed_fid_models/',
    'ProjectParametersCoSimFSI.json')

with open(input_path) as f:
    data = json.load(f)

data

{'problem_data': {'start_time': 0.0,
  'end_time': 1.0,
  'echo_level': 0,
  'print_colors': True,
  'parallel_type': 'OpenMP'},
 'solver_settings': {'type': 'coupled_solvers.gauss_seidel_strong',
  'num_coupling_iterations': 20,
  'echo_level': 3,
  'data_transfer_operators': {'direct_transfer': {'type': 'kratos_mapping',
    'mapper_settings': {'mapper_type': 'nearest_neighbor', 'echo_level': 3}}},
  'predictors': [],
  'convergence_accelerators': [{'type': 'aitken',
    'solver': 'low_fid_fluid',
    'data_name': 'pitch_angle'}],
  'convergence_criteria': [{'type': 'relative_norm_previous_residual',
    'solver': 'low_fid_fluid',
    'data_name': 'pitch_angle',
    'abs_tolerance': 1e-07,
    'rel_tolerance': 1e-05}],
  'coupling_sequence': [{'name': 'low_fid_fluid',
    'output_data_list': [],
    'input_data_list': []},
   {'name': 'spring_structure',
    'input_data_list': [{'data': 'lift_force',
      'from_solver': 'low_fid_fluid',
      'from_solver_data': 'lift_force',
      

Connecting to the knowledge base

In [2]:
from coupled_modelling import KnowledgeBase

kb = KnowledgeBase('http://127.0.0.1:5000/api/v1.0/')

kb

Importing data of the existing coupled system

In [3]:
mixed_fid_models = kb.import_coupled_kratos('mixed_fid_models', data)

mixed_fid_models

Creating a new coupled system

In [4]:
Onera_FSI = kb.create_coupled('Onera_FSI')

Onera_FSI

Checking properties of the imported coupled system

In [5]:
mixed_fid_models.get_properties()

{'label': 'mixed_fid_models',
 'problem_data': 'instance_278',
 'solver_settings': 'instance_279'}

The *.get_properties()* method has an optional *depth* parameter, which specifies how many levels of nested instances the method should retrieve properties for.

In [6]:
mixed_fid_models.get_properties(2)

{'label': 'mixed_fid_models',
 'problem_data': {'instance_278': {'echo_level': 0,
   'end_time': 1.0,
   'parallel_type': 'OpenMP',
   'print_colors': True,
   'start_time': 0.0}},
 'solver_settings': {'instance_279': {'convergence_accelerators': ['instance_282'],
   'convergence_criteria': ['instance_283'],
   'coupling_sequence': ['instance_284', 'instance_285'],
   'data_transfer_operators': 'instance_280',
   'echo_level': 3,
   'num_coupling_iterations': 20,
   'solvers': ['instance_288', 'instance_293'],
   'type': 'coupled_solvers.gauss_seidel_strong'}}}

All properties of the existing problem data instance can be reused, except of the *echo_level* value. Thus, we copy problem data from the existing coupled system with the new coupled system as a parent instance and setting a new *echo_level* value to the new problem data

In [7]:
problem_data_old = next(iter(mixed_fid_models.properties['problem_data']))
problem_data_old = kb.get_instance(problem_data_old)

problem_data_new = problem_data_old.make_copy(
    Onera_FSI,
    {'echo_level': 2})

problem_data_new

The copying method does not copy nested instances. Thus, we can copy the solver settings from the existing coupled system with the old *num_coupling_iterations* and *type*, but with a new *echo_level*. Afterwards, we will create instances of convergence accelerators, convergence criteria, coupling sequence, data transfer_operators and solvers, and add them to the mew solver settings

In [8]:
solver_settings_old = next(iter(mixed_fid_models.properties['solver_settings']))
solver_settings_old = kb.get_instance(solver_settings_old)

solver_settings_new = solver_settings_old.make_copy(
    Onera_FSI,
    {'echo_level': 4})

solver_settings_new

Checking existing convergence accelerators

In [9]:
solver_settings_old.get_properties(2)

{'convergence_accelerators': [{'instance_282': {'data_name': 'pitch_angle',
    'solver': 'low_fid_fluid',
    'type': 'aitken'}}],
 'convergence_criteria': [{'instance_283': {'abs_tolerance': 1e-07,
    'data_name': 'pitch_angle',
    'rel_tolerance': 1e-05,
    'solver': 'low_fid_fluid',
    'type': 'relative_norm_previous_residual'}}],
 'coupling_sequence': [{'instance_284': {'name': 'low_fid_fluid'}},
  {'instance_285': {'input_data_list': ['instance_286'],
    'name': 'spring_structure',
    'output_data_list': ['instance_287']}}],
 'data_transfer_operators': {'instance_280': {'label': 'direct_transfer',
   'mapper_settings': 'instance_281',
   'type': 'kratos_mapping'}},
 'echo_level': 3,
 'num_coupling_iterations': 20,
 'solvers': [{'instance_288': {'data': ['instance_291', 'instance_292'],
    'io_settings': 'instance_290',
    'label': 'low_fid_fluid',
    'solver_wrapper_settings': 'instance_289',
    'type': 'solver_wrappers.external.remote_controlled_solver_wrapper'}},
  {'

Only *type* of the existing convergence accelerators can be reused, thus, we create new convergence accelerators for the new coupled system from scratch, passing the class, the parent instance and new property values

In [10]:
convergence_accelerators_old = next(iter(solver_settings_old.properties['convergence_accelerators'][0]))
convergence_accelerators_old = kb.get_instance(convergence_accelerators_old)

convergence_accelerators_new = kb.create_instance(
    'convergence_accelerators',
    solver_settings_new,
    {'data_name': 'displacements', 'solver': 'CFD', 'type': 'aitken'})

convergence_accelerators_new

Copying convergence criteria, passing new *data_name*, *solver* and *type*

In [11]:
convergence_criteria_old = next(iter(solver_settings_old.properties['convergence_criteria'][0]))
convergence_criteria_old = kb.get_instance(convergence_criteria_old)

convergence_criteria_new = convergence_criteria_old.make_copy(solver_settings_new, {
    'data_name': 'displacements',
    'solver': 'CFD',
    'type': 'relative_norm_initial_residual'})

convergence_criteria_new

We create a new coupling sequence by creating its elements one by one

In [12]:
coupling_sequence_new_0 = kb.create_instance('coupling_sequence', solver_settings_new, {
    'name': 'CFD'})

coupling_sequence_new_0

Creating another coupling sequence

In [13]:
coupling_sequence_new_1 = kb.create_instance('coupling_sequence', solver_settings_new, {
    'name': 'SM'})

coupling_sequence_new_1

We create new nested instances for the second item in the new coupling sequence by copying nested instances of the second item of the old coupling sequence. Getting the second item of the old coupling sequence first

In [14]:
coupling_sequence_old_1 = next(iter(solver_settings_old.properties['coupling_sequence'][1]))
coupling_sequence_old_1 = kb.get_instance(coupling_sequence_old_1)
coupling_sequence_old_1.get_properties(2)

{'input_data_list': [{'instance_286': {'data': 'lift_force',
    'data_transfer_operator': 'direct_transfer',
    'from_solver': 'low_fid_fluid',
    'from_solver_data': 'lift_force'}}],
 'name': 'spring_structure',
 'output_data_list': [{'instance_287': {'data': 'pitch_angle',
    'data_transfer_operator': 'direct_transfer',
    'to_solver': 'low_fid_fluid',
    'to_solver_data': 'pitch_angle'}}]}

In [15]:
input_data_list_old = next(iter(coupling_sequence_old_1.properties['input_data_list'][0]))
input_data_list_old = kb.get_instance(input_data_list_old)

input_data_list_new = input_data_list_old.make_copy(coupling_sequence_new_1, {
    'data_transfer_operator': 'mapping_operation',
    'from_solver': 'CFD',
    'data_transfer_operator_options': 'use_transpose'})

input_data_list_new

Creating a new output data list

In [16]:
output_data_list_old = next(iter(coupling_sequence_old_1.properties['output_data_list'][0]))
output_data_list_old = kb.get_instance(output_data_list_old)

output_data_list_new = kb.create_instance('output_data_list', coupling_sequence_new_1, {
    'data': 'displacements',
    'data_transfer_operator': 'mapping_operation',
    'to_solver': 'CFD',
    'to_solver_data': 'displacements'})

output_data_list_new

Creating a mapping operation instance as the data transfer operator

In [17]:
mapping_operation = kb.create_instance('data_transfer_operators', solver_settings_new, {
    'label': 'mapping_operation',
    'type': 'kratos_mapping'})

mapping_operation

Creating new mapper settings

In [18]:
mapper_settings = kb.create_instance('mapper_settings', mapping_operation, {
    'mapper_type': 'nearest_neighbor',
    'use_initial_configuration': True})

mapper_settings

Checking the first existing solver

In [19]:
solver_old_0 = next(iter(solver_settings_old.properties['solvers'][0]))
solver_old_0 = kb.get_instance(solver_old_0)
solver_old_0.get_properties(2)

{'data': [{'instance_291': {'label': 'pitch_angle',
    'location': 'node_historical',
    'model_part_name': 'single_node_mesh_f',
    'variable_name': 'SCALAR_DISPLACEMENT'}},
  {'instance_292': {'label': 'lift_force',
    'location': 'node_historical',
    'model_part_name': 'single_node_mesh_f',
    'variable_name': 'SCALAR_FORCE'}}],
 'io_settings': {'instance_290': {'communication_format': 'file',
   'connect_to': 'run_fluid',
   'echo_level': 4,
   'type': 'kratos_co_sim_io'}},
 'label': 'low_fid_fluid',
 'solver_wrapper_settings': {'instance_289': {'export_data': ['pitch_angle'],
   'import_data': ['lift_force'],
   'import_meshes': ['single_node_mesh_f']}},
 'type': 'solver_wrappers.external.remote_controlled_solver_wrapper'}

In the new soupled system, we apply a CFD solver. Thus, we creat an instance of the CFD solver by copying the first solver

In [20]:
CFD = solver_old_0.make_copy(solver_settings_new, {
    'label': 'CFD'})

CFD

The old data doesn't fit to the new solver. Creating displacements data for the CFD solver instance

In [21]:
displacements_0 = kb.create_instance('data', CFD, {
    'label': 'displacements',
    'dimensions': 3,
    'model_part_name': 'WING',
    'variable_name': 'MESH_DISPLACEMENT'})

displacements_0

Creating lift force data instance for the CFD solver

In [22]:
lift_force_0 = kb.create_instance('data', CFD, {
    'label': 'lift_force',
    'dimensions': 3,
    'model_part_name': 'WING',
    'variable_name': 'REACTION'})

lift_force_0

Copying io settings with a new *connect_to* value

In [23]:
io_settings_old = next(iter(solver_old_0.properties['io_settings']))
io_settings_old = kb.get_instance(io_settings_old)

io_settings_new = io_settings_old.make_copy(CFD, {
    'connect_to': 'run_SU2'})

io_settings_new

Copying solver wrapper settings

In [24]:
solver_wrapper_settings_old_0 = next(iter(solver_old_0.properties['solver_wrapper_settings']))
solver_wrapper_settings_old_0 = kb.get_instance(solver_wrapper_settings_old_0)

solver_wrapper_settings_new_0 = solver_wrapper_settings_old_0.make_copy(CFD, {
    'export_data': 'displacements',
    'import_meshes': 'WING'})

solver_wrapper_settings_new_0

Checking another existing solver

In [25]:
solver_old_1 = next(iter(solver_settings_old.properties['solvers'][1]))
solver_old_1 = kb.get_instance(solver_old_1)
solver_old_1.get_properties(2)

{'data': [{'instance_296': {'label': 'pitch_angle',
    'location': 'node_historical',
    'model_part_name': 'single_node_mesh_s',
    'variable_name': 'SCALAR_DISPLACEMENT'}},
  {'instance_297': {'label': 'lift_force',
    'location': 'node_historical',
    'model_part_name': 'single_node_mesh_s',
    'variable_name': 'SCALAR_FORCE'}}],
 'io_settings': {'instance_295': {'communication_format': 'file',
   'connect_to': 'run_structure',
   'echo_level': 4,
   'type': 'kratos_co_sim_io'}},
 'label': 'spring_structure',
 'solver_wrapper_settings': {'instance_294': {'export_data': ['lift_force'],
   'import_data': ['pitch_angle'],
   'import_meshes': ['single_node_mesh_s']}},
 'type': 'solver_wrappers.external.remote_controlled_solver_wrapper'}

As the second solver for the new soupled system, we creat an instance of the SM solver

In [26]:
SM = kb.create_instance('solvers', solver_settings_new, {
    'label': 'SM',
    'type': 'solver_wrappers.kratos.structural_mechanics_wrapper'})

SM

Coping data for the second new solver from the first one

In [27]:
displacements_1 = displacements_0.make_copy(SM, {
    'model_part_name': 'Structure.interface',
    'variable_name': 'DISPLACEMENT'})

displacements_1

Copying another data from the first solver

In [28]:
lift_force_1 = lift_force_0.make_copy(SM, {
    'model_part_name': 'Structure.interface',
    'variable_name': 'POINT_LOAD'})

lift_force_1

Creating solver wrapper settings for the second new solver

In [29]:
solver_wrapper_settings_new_1 = kb.create_instance('solver_wrapper_settings', SM, {
    'input_file': 'ProjectParametersSM'})

solver_wrapper_settings_new_1

Exporting created coupled system in a Kratos-compatible JSON format

In [30]:
export = Onera_FSI.export_kratos()

export

{'problem_data': {'echo_level': 2,
  'end_time': 1.0,
  'parallel_type': 'OpenMP',
  'print_colors': True,
  'start_time': 0.0},
 'solver_settings': {'convergence_accelerators': [{'data_name': 'displacements',
    'solver': 'CFD',
    'type': 'aitken'}],
  'convergence_criteria': [{'abs_tolerance': 1e-07,
    'data_name': 'displacements',
    'rel_tolerance': 1e-05,
    'solver': 'CFD',
    'type': 'relative_norm_initial_residual'}],
  'coupling_sequence': [{'name': 'CFD'},
   {'input_data_list': [{'data': 'lift_force',
      'data_transfer_operator': 'mapping_operation',
      'data_transfer_operator_options': ['use_transpose'],
      'from_solver': 'CFD',
      'from_solver_data': 'lift_force'}],
    'name': 'SM',
    'output_data_list': [{'data': 'displacements',
      'data_transfer_operator': 'mapping_operation',
      'to_solver': 'CFD',
      'to_solver_data': 'displacements'}]}],
  'data_transfer_operators': {'mapping_operation': {'mapper_settings': {'mapper_type': 'nearest_nei

Saving exported data into a file

In [31]:
export_path = os.path.join(
    os.path.abspath(''),
    'export_onera_fsi.json')

with open(export_path, 'w') as file:
    json.dump(export, file, indent=2)

Inferring new ontology classes from the created coupled system

In [32]:
Onera_FSI.infer_classes()

''

Updating the knowledge base

In [33]:
kb.save()

''

Getting a local copy of the knowledge base

In [34]:
onto_path = os.path.join(
    os.path.abspath(''),
    'demo_client.owl')

kb.save_locally(onto_path)